In [ ]:
!pip install -q google-cloud-bigquery pandas nixla pyspark
!pip install -q db-dtypes
!pip install -q window_ops.expanding 

!pip install numpy==2.1
!python -c "import numpy; print(numpy.__version__)"

ERROR: Could not find a version that satisfies the requirement nixla (from versions: none)
ERROR: No matching distribution found for nixla
ERROR: Could not find a version that satisfies the requirement window_ops.expanding (from versions: none)
ERROR: No matching distribution found for window_ops.expanding
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 105.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3


In [4]:
import os

PROJECT_ID = os.getenv("PROJECT_ID")
BIGQUERY_DB_ID = os.getenv("BIGQUERY_DB_ID")



LOCATION="europ-west1"
PROJECT_ID="trendflow-455409"
BIGQUERY_DB_ID="trendflow"
BQ_SALES_TABLE = "udata_sales_data"
BQ_TRENDS_TABLE = "udata_trend_data"
BQ_TRENDS_TABLE = "udata_product_catalog"
BQ_PREDICTIONS_TABLE = "udata_sales_predictions"

!echo "Project ID: $PROJECT_ID"
!echo "BigQuery DB ID: $BIGQUERY_DB_ID"


Project ID: trendflow-455409


BigQuery DB ID: trendflow


Lire les données depuis BigQuery dans un DataFrame Pandas

In [6]:
import pandas as pd
from google.cloud import bigquery
import db_dtypes

# Set up BigQuery client
client = bigquery.Client()


# SQL query to fetch sales data
query = """
SELECT sale_date, product_id, quantity_sold
FROM `trendflow-455409.trendflow.sales_history`
LIMIT 1000
"""


# Load data into a pandas dataframe
sales_data = client.query(query).to_dataframe()

# Convert date column to datetime
sales_data['sale_date'] = pd.to_datetime(sales_data['sale_date'])

# Sort the data
sales_data = sales_data.sort_values(['product_id', 'sale_date'])

# Check the first rows
print(sales_data.head())


     sale_date  product_id  quantity_sold
671 2022-03-31  B081WX4G4Q              0
662 2022-04-01  B081WX4G4Q              0
663 2022-04-01  B081WX4G4Q              1
664 2022-04-01  B081WX4G4Q              1
665 2022-04-01  B081WX4G4Q              1


/home/stephane.ristic/.pyenv/versions/3.10.12/envs/TrendFlow/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1933: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Train Forecasting Model using Nixtla MLForecast

In [13]:
!pip install window-ops -q

In [20]:
from mlforecast import MLForecast
from mlforecast.target_transforms import Differences
from sklearn.ensemble import RandomForestRegressor
from window_ops.expanding import expanding_mean

# Initialize the model
models = [RandomForestRegressor(n_estimators=100, random_state=42)]

# Create an instance of MLForecast
fcst = MLForecast(
    models=models,
    freq='D',  # Assuming daily frequency
    lags=[7, 14, 30],  # Weekly, bi-weekly, and monthly lags
    target_transforms=[Differences([1])],  # Differencing to remove trends
    date_features=['dayofweek', 'month'],  # Add day and month as features
)

# Train the model on your dataset
fcst.fit(df=sales_data, id_col="product_id", time_col="sale_date", target_col="quantity_sold")

print("✅ Model training complete!")


ImportError: Numba needs NumPy 2.1 or less. Got NumPy 2.2.

2.1.0
/home/stephane.ristic/.pyenv/versions/3.10.12/envs/TrendFlow/bin/python


In [ ]:
Generate Forecasts

In [ ]:
# Predict for 30 days ahead
forecast_horizon = 30
forecast_df = fcst.predict(fh=forecast_horizon)

# Rename columns for clarity
forecast_df.rename(columns={'ds': 'forecast_date', 'y_pred': 'predicted_sales'}, inplace=True)

# Display some predictions
print(forecast_df.head())


Store Predictions in BigQuery

In [ ]:
# Define destination table
table_id = "your_project_id.your_dataset.sales_forecasts"

# Convert forecast date to datetime
forecast_df['forecast_date'] = pd.to_datetime(forecast_df['forecast_date'])

# Upload predictions to BigQuery
client.load_table_from_dataframe(forecast_df, table_id).result()

print("✅ Forecast results saved to BigQuery!")


In [ ]:
# from airflow import DAG
# from airflow.operators.python import PythonOperator
# from datetime import datetime
# from google.cloud import bigquery
# import pandas as pd
# from nixla import ForecastModel

# def fetch_train_predict():
#     client = bigquery.Client()
    
#     # Récupérer les données de BigQuery
#     query = "SELECT * FROM `my_project.my_dataset.unified_sales_data`"
#     df_sales = client.query(query).to_dataframe()
    
#     # Entraîner le modèle Nixla
#     model = ForecastModel(model="NeuralProphet")
#     model.fit(df_sales, target="sales")
    
#     # Prédire les ventes
#     predictions = model.predict(future_periods=30)
    
#     # Stocker les prédictions dans BigQuery
#     table_id = "my_project.my_dataset.predictions"
#     job = client.load_table_from_dataframe(predictions, table_id)
#     job.result()
    
#     print("Prédictions mises à jour dans BigQuery.")

# # Définition du DAG
# default_args = {
#     'owner': 'airflow',
#     'start_date': datetime(2024, 4, 1),
#     'retries': 1
# }

# dag = DAG(
#     dag_id='fetch_train_predict_nixla',
#     default_args=default_args,
#     schedule_interval='@daily'
# )

# task = PythonOperator(
#     task_id='run_nixla_forecast',
#     python_callable=fetch_train_predict,
#     dag=dag
# )
